## IMPORTING ESSENTIAL LIBARARIES

In [ ]:
print('Importing Essential Libraries...')

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import os
import sklearn
%matplotlib inline

print('Done!!!')

## Importing classes from sklearn 

In [ ]:
#Importing the sklean classes


from sklearn.linear_model import LogisticRegression
logistic_Regression  = LogisticRegression(max_iter=10000)

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score,accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

from collections import Counter


In [ ]:
pd.options.display.max_columns = 500

## Reading the data

In [ ]:
print('Loading the data.....')
df = pd.read_csv('./Data/train_data.csv')
print('Done!!!')

In [ ]:
print('The first 5 rows are: ')
df.head()

In [ ]:
print('The data has the following information: ')
df.info()

In [ ]:
print('The shape of the dataframe is:- Rows: ',df.shape[0],' Columns: ',df.shape[1])

## Data visualization and Data Analytics

In [ ]:
#In the given data how many loans replayed and defaulted loans.
# fig, ax = plt.subplots(figsize=(7,5))
# sns.countplot(x = 'TARGET',data = df)

In [ ]:
#Who is the highest borrower? Male or Female?
# fig, ax = plt.subplots(figsize=(10,7))
# sns.countplot(x='CODE_GENDER',data=df)

In [ ]:
# #How is the distribution of target labels? - Did most people return on time ?
# fig, ax = plt.subplots(figsize=(10,7))
# sns.countplot(x ='TARGET',data=df, hue='TARGET',palette="Set1")

In [ ]:
# #Whether is it Female who has more difficulties or is it Male in repaying the loan?
# fig, ax = plt.subplots(figsize=(10,7))
# sns.countplot(x='TARGET',hue='CODE_GENDER',data=df)

In [ ]:
# #Who owns most number of the cars? M or F?
# fig, ax = plt.subplots(figsize=(10,7))
# sns.countplot(x='CODE_GENDER', hue='FLAG_OWN_CAR', data=df,palette="Set1")

In [ ]:
l1 = []
l2 = []

for i in df.columns:
    if df[i].nunique() < 100:
        l1.append(i)
    else:
        l2.append(i)
print(l1)
print(l2)

In [ ]:
# print('Drawing Histogram!!!')
# for h in l2:
#     fig, ax = plt.subplots(1,1, figsize=(15, 6))
#     sns.histplot(df[h], palette='Blues_r')
#     fig.text(0.1, 0.95, f'{h}', fontsize=16, fontweight='bold', fontfamily='serif')
#     plt.xlabel('value ', fontsize=10)
#     plt.ylabel('count',fontsize=10)
#     plt.yticks(fontsize=13)
#     plt.box(False)


## PREPROCESSING THE ENTIRE DATASET

### FINDING THE COLUMNS THAT HAVE NULL VALUES OF MORE THAN 50%
### DROP THEM

In [ ]:
to_drop=[] #this is a list that stores the names of cols having more than 50% nulls
for features in df.columns:
    percentage = (df[features].isna().sum()/df.shape[0]) *100
    if df[features].isna().sum() > 0 and percentage > 50.0:
        to_drop.append(features)
        print(features,'    ' ,df[features].isna().sum(), percentage)
        df.drop(features,axis=1,inplace=True)

In [ ]:
print('The shape of the dataframe after deleting the columns are:- Rows: ',df.shape[0],' Columns: ',df.shape[1])

In [ ]:
#Dropping the SK_ID_CURR column as it a primary key and has no actual use here.
df.drop('SK_ID_CURR',axis= 1,inplace= True)

In [ ]:
#Check for duplicate data rows
countDuplicateRows = df[df.duplicated(subset = None, keep= False)].shape[0]
print('The number of Duplicate Rows present here are: ',countDuplicateRows)

## TAKING CARE OF NULL VALUES IN THE REMAINING COLUMNS

In [ ]:
#We have deleted the columns with >50% Null values, but there are some columns with null values. 
#Handling them here.
#Checking the remaining columns with null values and their corresponding median and mean values and store them in a list.

count = 0
featureList = []
for features in df.columns:
    if df[features].isna().sum() > 0 and df[features].dtype != object:
        featureList.append(features)
        print(features, '   ', df[features].isna().sum(),'  ',df[features].median(), '  ',df[features].mean() )
        count += 1
print('\n\nTotal such columns are',count)

In [ ]:
#Printing the List that we got here.(i.e names of features having some null values)
print(featureList)

In [ ]:
#From the list obtained in the previous cell, we have to pick out those who have integer numbers and then fill the rest i.e float values. 
#ones with median values of the entire column.


modifiedListFeaturesWithWholeNumbers = ['CNT_FAM_MEMBERS','OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE','AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR']

s = set()
for i in modifiedListFeaturesWithWholeNumbers:
    s.add(i)

#Contains only the list with floating point values.
newList = []
for i in featureList:
    if i not in s:
       newList.append(i) 
print(newList)


In [ ]:
#Replace the null values in columns with floating point values and fill it out with median of the entire column.
for i in newList:
    df[i] = df[i].fillna(df[i].median())

In [ ]:
# The left out columns of the categorical type and integer numbers.

for i in df.columns:
    if df[i].isna().sum() > 0:
        print(i,df[i].dtype)


In [ ]:
#Fill the elements in modifiedListFeaturesWithWholeNumbers with integers.
for i in modifiedListFeaturesWithWholeNumbers:
    if df[i].dtype != object:
        df[i] = df[i].fillna(int(df[i].mean()))

In [ ]:
#The columns left out with still null values. These are mainly of type string/object.
categoricalData = []
for i in df.columns:
    if df[i].isna().sum() > 0:
        print(i,df[i].dtype,df[i].isna().sum())
        categoricalData.append(i)

In [ ]:
#Finding the Columns that have values that don't make any sense.
for i in df.columns:
    if df[i].dtype == 'object':
        print(i,'   ',df[i].unique(),'  ',df[i].isna().sum())

In [ ]:
# These columns have values that don't make any sense.

print(df['CODE_GENDER'].unique())               # XNA
print(df['NAME_TYPE_SUITE'].unique())           # Other_A, Other_B
print(df['NAME_FAMILY_STATUS'].unique())        # Unknown
print(df['ORGANIZATION_TYPE'].unique())         # XNA

In [ ]:
#Replacing these Absurd values with NaN values.

df['CODE_GENDER'] = df['CODE_GENDER'].replace('XNA',np.nan)
df['NAME_TYPE_SUITE'] = df['NAME_TYPE_SUITE'].replace('Other_A',np.nan)
df['NAME_TYPE_SUITE'] = df['NAME_TYPE_SUITE'].replace('Other_B',np.nan)
df['NAME_FAMILY_STATUS'] = df['NAME_FAMILY_STATUS'].replace('Unknown',np.nan)
df['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace('XNA',np.nan)

In [ ]:
#Handling the nulls in categorical data.
for i in df.columns:
    if df[i].dtype == 'object':
        df[i] = df[i].fillna(df[i].mode()[0])

#NULLS HAVE BEEN REMOVED.
df.isna().sum()

In [ ]:
#DAYS_LAST_PHONE_CHANGE      #contians negative date values

#Need to handle this also. Can't understand what this -ve value means.


## LABEL ENCODING FOR CATEGORICAL DATA

In [ ]:
#Label Encoding all the categorical object using Label Encoder.
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

print('Applying Label Encoding....')
for i in df.columns:
    if df[i].dtype == 'object':
        df[i] = label_encoder.fit_transform(df[i])
print('Done!!!')

In [ ]:
x = df.drop('TARGET',axis = 1)
y = df['TARGET']

In [ ]:
df['TARGET'].value_counts()

### HANDLING THE IMBALANCED DATASET

In [ ]:
import imblearn
from imblearn.over_sampling import KMeansSMOTE,SMOTE,ADASYN,SVMSMOTE
ksmote = ADASYN(0.75,random_state=42)
X_res,Y_res = ksmote.fit_resample(x,y)


print("The number of classes before fit {}",format(Counter(y)))
print("The number of classes after fit {}",format(Counter(Y_res)))

print('Shape before sampling',x.shape,y.shape)
print('Shape after sampling',X_res.shape,Y_res.shape)


### TRAIN TEST SPLIT THE ENTIRE DATA

In [ ]:
from sklearn.model_selection import train_test_split


X_Train,X_Test,Y_Train,Y_Test = train_test_split(X_res,Y_res,train_size= 0.7,shuffle= True,random_state=42)
print("Training Data shape:  ",X_Train.shape,Y_Train.shape)
print("Testing  Data shape:  ",X_Test.shape,Y_Test.shape)

## Finding the correlation of the inpendent features with the output.

In [ ]:


abs_corr_matrix = X_Train.corr().abs()                               #absoulte value correlation matrix
corr_target_checker = abs_corr_matrix[len(abs_corr_matrix)-1:]  #corr of target
corr_target_checker
to_drop=[]                                                      # A list that stores col names of low corr
count = 0
for i in corr_target_checker.columns:
    if corr_target_checker[i].sum() < 0.0005:
        print('Value: ',corr_target_checker[i].sum(),'Feature Name: ',i)
        to_drop.append(i)
        count += 1
print(count)
print(to_drop)

print('Dropping Columns....')
print(X_Train.shape)
X_Train.drop(to_drop,axis= 1,inplace= True)
print(X_Train.shape)
print('Done!!')

### FEATURE SELECTION USING RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel


# For Random Forest Classifier
sel = SelectFromModel(RandomForestClassifier(n_estimators= 100,criterion= 'entropy'))
sel.fit(X_Train,Y_Train)


# # For Decision Tree Classifier
# sel = SelectFromModel(ExtraTreesClassifier(n_estimators= 100,criterion= 'gini'))
# sel.fit(X_Train,Y_Train)


selected_features = X_Train.columns[(sel.get_support())]
print("The Number of features selected are: ",len(selected_features))
print("The features selected are: ",selected_features)

# pd.series(sel.estimator_,feature_importa).hist()

#Dropping the columns that are not present in the selected_features list

for i in X_Train.columns:
    if i not in selected_features:
        X_Train.drop(i,axis= 1,inplace= True)


In [ ]:
#Storing the selected_features for doing EDA on the Test Data.
%store selected_features

In [ ]:
print(X_Train.columns)

## Removing Outliers Using IQR

In [ ]:
# #Plotting BoxPlot of the left out columns to check if they have any outliers.
# print('Before Removing Outliers')
# count = 0
# for i in X_Train.columns:
#     fig = plt.figure(figsize=(7,4))
#     plt.boxplot(X_Train[i])
#     plt.suptitle(i)
#     plt.show()
#     count += 1

# print('Total Boxplots printed are: ',count)


In [ ]:
#Now find the outliers using IQR Method and remove them from the entire data X_Train
def drop_outliers_IQR(df,feature):
    iqr = 1.5*( np.percentile(df[feature],75) - np.percentile(df[feature],25)   )
    # df.drop( df[df[feature] > (iqr + np.percentile(df[feature],75))].index,inplace= True )
    # df.drop( df[df[feature] < (np.percentile(df[feature],25) - iqr)].index,inplace= True )

    df.loc[df[feature] > (iqr + np.percentile(df[feature],75))] = df[feature].mean()
    df.loc[df[feature] < (np.percentile(df[feature],25) - iqr)] = df[feature].mean()
    #df[feature] = df[feature].fillna(df[feature].mean())
   

for i in X_Train.columns:
    drop_outliers_IQR(X_Train,i)


# drop_outliers_IQR(X_Train,'CNT_CHILDREN')
# drop_outliers_IQR(X_Train,'NAME_FAMILY_STATUS')
# drop_outliers_IQR(X_Train,'NAME_HOUSING_TYPE')
# drop_outliers_IQR(X_Train,'OCCUPATION_TYPE')
# drop_outliers_IQR(X_Train,'CNT_FAM_MEMBERS')
# drop_outliers_IQR(X_Train,'HOUR_APPR_PROCESS_START')
# drop_outliers_IQR(X_Train,'EXT_SOURCE_1')
# drop_outliers_IQR(X_Train,'EXT_SOURCE_3')
# drop_outliers_IQR(X_Train, 'ENTRANCES_MODE')
# drop_outliers_IQR(X_Train,'FLOORSMAX_MODE')
# drop_outliers_IQR(X_Train,'FLOORSMAX_MEDI')
# drop_outliers_IQR(X_Train,'WALLSMATERIAL_MODE')
# drop_outliers_IQR(X_Train,'DEF_30_CNT_SOCIAL_CIRCLE')
# drop_outliers_IQR(X_Train,'OBS_60_CNT_SOCIAL_CIRCLE')
# drop_outliers_IQR(X_Train,'DEF_60_CNT_SOCIAL_CIRCLE')
# drop_outliers_IQR(X_Train,'AMT_REQ_CREDIT_BUREAU_DAY')
# drop_outliers_IQR(X_Train,'AMT_REQ_CREDIT_BUREAU_WEEK')
# drop_outliers_IQR(X_Train,'AMT_REQ_CREDIT_BUREAU_QRT')
# drop_outliers_IQR(X_Train,'AMT_REQ_CREDIT_BUREAU_YEAR')


In [ ]:
# #Plotting BoxPlot of the left out columns after removal of outliers.
# print('After Removing Outliers')
# count = 0
# for i in X_Train.columns:
#     fig = plt.figure(figsize=(7,4))
#     plt.boxplot(X_Train[i])
#     plt.suptitle(i)
#     plt.show()
#     count += 1

# print('Total Boxplots printed are: ',count)

In [ ]:
print(X_Train.shape,Y_Train.shape)

### REMOVING THE COLUMNS FROM THE TEST DATA PART

In [ ]:
for i in X_Test.columns:
    if i not in selected_features:
        X_Test.drop(i,axis=1,inplace=True)


## SCALING THE ENTIRE DATA USING STANDARD SCALAR

In [ ]:
print('Applying Scaling on the training data only for the features...')
scaler = StandardScaler()
scaler.fit(X_Train)
X_Train_scaled = scaler.transform(X_Train)
print('Done!!')
#Pass this scaled data as input to the Logistic Regression.

## IMPLEMENTING LOGISTIC REGRESSION

In [ ]:
logistic_Regression.fit(X_Train,Y_Train)
Y_Pred = logistic_Regression.predict(X_Test)

lacc = accuracy_score(Y_Pred,Y_Test)
lf1 = f1_score(Y_Pred,Y_Test)
lauc_score = roc_auc_score(Y_Pred,Y_Test)
print('The accuracy of the model on training data is: ')

print('The accuracy  is: ',lacc*100,'%')
print('The value of f1_score is: ',lf1*100,'%')
print('The value of Roc AUC Score is: ',lauc_score*100,'%')

print("The confusion matrix is: \n\n",confusion_matrix(Y_Test,Y_Pred))
print("The classification report is: \n\n",classification_report(Y_Test,Y_Pred))


## IMPLEMENTING K NEAREST NEIGHBOURS

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier

# for k in range(1, 100, 5):
#     k = k + 1
#     knn = KNeighborsClassifier(n_neighbors = k).fit(X_Train,Y_Train)
#     acc = knn.score(X_Test,Y_Test)
#     print("Accuracy for k = ",k," is: ",acc)

# Here we are selecting which is the best n value for the KNN algo

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors= 2).fit(X_Train,Y_Train)
Y_Pred = knn.predict(X_Test)
kacc = accuracy_score(Y_Pred,Y_Test)
kf1 = f1_score(Y_Pred,Y_Test)
kauc_score = roc_auc_score(Y_Pred,Y_Test)


print('The accuracy of the model on Data is: ')

print('The accuracy  is: ',kacc*100,'%')
print('The value of f1_score is: ',kf1*100,'%')
print('The value of Roc AUC Score is: ',kauc_score*100,'%')

print("The confusion matrix is: \n\n",confusion_matrix(Y_Test,Y_Pred))
print("The classification report is: \n\n",classification_report(Y_Test,Y_Pred))


## IMPLEMENTING SUPPORT VECTOR MACHINES(SVM)

In [ ]:
# from sklearn.svm import SVC


# svc = SVC()
# svc.fit(X_Train,Y_Train)

# Y_Pred = svc.predict(X_Test)
# acc = accuracy_score(Y_Pred,Y_Test)
# f1 = f1_score(Y_Pred,Y_Test)
# auc_score = roc_auc_score(Y_Pred,Y_Test)


# print('The accuracy of the model on Data is: ')

# print('The accuracy  is: ',acc*100,'%')
# print('The value of f1_score is: ',f1*100,'%')
# print('The value of Roc AUC Score is: ',auc_score*100,'%')

# print("The confusion matrix is: \n\n",confusion_matrix(Y_Test,Y_Pred))
# print("The classification report is: \n\n",classification_report(Y_Test,Y_Pred))




## IMPLEMENTING XGBOOST 

In [ ]:
from xgboost import XGBClassifier

#### applying Grid Search CV to get the best hyperparameters for the XGBoost classifier

In [ ]:
# xgb_model = XGBClassifier(random_state = 30)
# search_space = {
#     "n_estimators" :   [100,200],
#     "max_depth" :      [3,6,7],
#     "gamma" :          [0.01,0.1],
#     "learning_rate" :  [0.001,0.01,0.1,1]
# }

# from sklearn.model_selection import GridSearchCV
# GS = GridSearchCV(
#     estimator= xgb_model,
#     param_grid= search_space,
#     scoring= ["roc_auc","roc_auc_ovr","roc_auc_ovo","f1","f1_micro","f1_macro","accuracy"],
#     refit= "roc_auc",
#     cv= 5,
#     verbose= 4
# )

# GS.fit(X_Train,Y_Train)

# print("The best estimator is: ",GS.best_estimator_)
# print("The best parameter is: ",GS.best_params_)
# print("The best AUC_ROC score is: ",GS.best_score_)
# df_XGBoost = pd.DataFrame(GS.cv_results_)
# df_XGBoost = df_XGBoost.sort_values("rank_test_roc_auc")
# df_XGBoost.to_csv('./Test_Output/XGBoost_GridSearchCV.csv')

In [ ]:
model_XGB = XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.1, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.1, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=7, max_leaves=0, min_child_weight=1,
              monotone_constraints='()', n_estimators=200, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=30)
model_XGB.fit(X_Train,Y_Train)
Y_Pred = model_XGB.predict(X_Test)
xacc = accuracy_score(Y_Pred,Y_Test)
xf1 = f1_score(Y_Pred,Y_Test)
xauc_score = roc_auc_score(Y_Pred,Y_Test)


print('The accuracy of the model on Data is: ')

print('The accuracy  is: ',xacc*100,'%')
print('The value of f1_score is: ',xf1*100,'%')
print('The value of Roc AUC Score is: ',xauc_score*100,'%')

print("The confusion matrix is: \n\n",confusion_matrix(Y_Test,Y_Pred))
print("The classification report is: \n\n",classification_report(Y_Test,Y_Pred))


## IMPLEMENTING DECISION TREES

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_Train,Y_Train)
Y_Pred = dt.predict(X_Test)
dacc = accuracy_score(Y_Pred,Y_Test)
df1 = f1_score(Y_Pred,Y_Test)
dauc_score = roc_auc_score(Y_Pred,Y_Test)


print('The accuracy of the model on Data is: ')

print('The accuracy  is: ',dacc*100,'%')
print('The value of f1_score is: ',df1*100,'%')
print('The value of Roc AUC Score is: ',dauc_score*100,'%')

print("The confusion matrix is: \n\n",confusion_matrix(Y_Test,Y_Pred))
print("The classification report is: \n\n",classification_report(Y_Test,Y_Pred))


## IMPLEMENTING RANDOM FOREST

In [ ]:
rf = RandomForestClassifier().fit(X_Train,Y_Train)
Y_Pred = rf.predict(X_Test)
racc = accuracy_score(Y_Pred,Y_Test)
rf1 = f1_score(Y_Pred,Y_Test)
rauc_score = roc_auc_score(Y_Pred,Y_Test)


print('The accuracy of the model on Data is: ')

print('The accuracy  is: ',racc*100,'%')
print('The value of f1_score is: ',rf1*100,'%')
print('The value of Roc AUC Score is: ',rauc_score*100,'%')

print("The confusion matrix is: \n\n",confusion_matrix(Y_Test,Y_Pred))
print("The classification report is: \n\n",classification_report(Y_Test,Y_Pred))


## IMPLEMENTING VARIOUS NAIVE BAYES ALGO

In [ ]:
##GAUSSIAN NAIVE BAYES

from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_Train,Y_Train)
Y_Pred = model.predict(X_Test)
gnbacc = accuracy_score(Y_Pred,Y_Test)
gnbf1 = f1_score(Y_Pred,Y_Test)
gnbauc_score = roc_auc_score(Y_Pred,Y_Test)


print('The accuracy of the model on Data is: ')

print('The accuracy  is: ',gnbacc*100,'%')
print('The value of f1_score is: ',gnbf1*100,'%')
print('The value of Roc AUC Score is: ',gnbauc_score*100,'%')

print("The confusion matrix is: \n\n",confusion_matrix(Y_Test,Y_Pred))
print("The classification report is: \n\n",classification_report(Y_Test,Y_Pred))


In [ ]:
## BERNOULLI NAIVE BAYES

from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB()
model.fit(X_Train,Y_Train)
Y_Pred = model.predict(X_Test)
bnbacc = accuracy_score(Y_Pred,Y_Test)
bnbf1 = f1_score(Y_Pred,Y_Test)
bnbauc_score = roc_auc_score(Y_Pred,Y_Test)


print('The accuracy of the model on Data is: ')

print('The accuracy  is: ',bnbacc*100,'%')
print('The value of f1_score is: ',bnbf1*100,'%')
print('The value of Roc AUC Score is: ',bnbauc_score*100,'%')

print("The confusion matrix is: \n\n",confusion_matrix(Y_Test,Y_Pred))
print("The classification report is: \n\n",classification_report(Y_Test,Y_Pred))


## COMPARING ALL THE MODELS

In [ ]:
from prettytable import PrettyTable


t = PrettyTable(['ALGO','ACCURACY SCORE','F1 SCORE','ROC-AUC SCORE'])
t.add_row(['Logistic Regression',round(lacc,2),round(lf1,2),round(lauc_score,2)])
t.add_row(['KNN',round(kacc,2),round(kf1,2),round(kauc_score,2)])
t.add_row(['XG Boost',round(xacc,2),round(xf1,2),round(xauc_score,2)])
t.add_row(['Decision Trees',round(dacc,2),round(df1,2),round(dauc_score,2)])
t.add_row(['Random Forest',round(racc,2),round(rf1,2),round(rauc_score,2)])
t.add_row(['Gaussian NB', round(gnbacc,2),round(gnbf1,2),round(gnbauc_score,2)])
t.add_row(['Bernoulli NB',round(bnbacc,2),round(bnbf1,2),round(bnbauc_score,2)])


print(t)

#Looking at the values it can be concluded that the best algo is XGBOOST!!!!

### IMPORTING THE TEST DATASET

In [ ]:
print('Loading the test data.....')
df_test = pd.read_csv('./Data/test_data.csv')
print('Done!!!')

In [ ]:
df_test.head()

In [ ]:
df_test.info()

In [ ]:
id_column = df_test['SK_ID_CURR']

### Dropping the unnecessary columns from test dataset

In [ ]:
# Dropping all the columns that are not in X_Train
for i in df_test.columns:
    if i not in selected_features:
        df_test.drop(i,axis=1,inplace= True)

print(df_test.shape)


### Removing the Unnecessary values

In [ ]:
for i in df_test.columns:
    if df_test[i].dtype == 'object':
        print("Column: ",i," Values: ",df_test[i].unique())

df_test['ORGANIZATION_TYPE'] = df['ORGANIZATION_TYPE'].replace('XNA',np.nan)

### Handling the NULL Values

In [ ]:
for i in df_test.columns:
    if df_test[i].dtype == 'object':
        df_test[i] = df_test[i].fillna(df_test[i].mode()[0])
    if df_test[i].dtype == 'int64':
        df_test[i] = df_test[i].fillna(int(df_test[i].mean()))
    if df_test[i].dtype == 'float64':
        df_test[i] = df_test[i].fillna(df_test[i].median())

### Applying the Label Encoding

In [ ]:
#Segregating the categorical and non-categorical data
cat = []
for i in df_test.columns:
    if df_test[i].dtype == 'object':
        cat.append(i)

from sklearn import preprocessing
label_encoder_test = preprocessing.LabelEncoder()

print('Applying Label Encoding to only Categorical Data....')
for i in cat:
    df_test[i] = label_encoder_test.fit_transform(df_test[i])

print('Done!!!')



### Apply OutLier Removing from non categorical columns in Test Data

In [ ]:
def drop_outliers_IQR(df,feature):
    iqr = 1.5*( np.percentile(df[feature],75) - np.percentile(df[feature],25)   )
    # df.drop( df[df[feature] > (iqr + np.percentile(df[feature],75))].index,inplace= True )
    # df.drop( df[df[feature] < (np.percentile(df[feature],25) - iqr)].index,inplace= True )

    df.loc[df[feature] > (iqr + np.percentile(df[feature],75))] = df[feature].mean()
    df.loc[df[feature] < (np.percentile(df[feature],25) - iqr)] = df[feature].mean()
    #df[feature] = df[feature].fillna(df[feature].mean())
   

for i in df_test.columns:
    if i not in cat:
        drop_outliers_IQR(df_test,i)

##### The test data is ready and could be now applied to the model.

### Lets apply the best model for which we got the highest AUC_ROC Score i.e XGBoost

In [ ]:
Y_Pred_final = model_XGB.predict(df_test)

# Here Y_Pred_final is the final data prepared by us now we need to upload it to  kaggle.


print(type(Y_Pred_final))
print(type(id_column))


## Exporting the Data for Kaggle Competition

In [ ]:
df_kaggle = id_column.to_frame()

df_kaggle['TARGET'] = Y_Pred_final.tolist()

df_kaggle.to_csv('./Test_Output/Submission_XGB.csv',index= None)